In [14]:
import cv2
import math
import time
import numpy as np
import HandTrackingModule as htm
import serial
import serial.tools.list_ports

In [15]:
# Global parameters
wCam, hCam = 640, 840
pTime = 0  # Initiate previous time for FPS calculation
angle = 0
angle_bar = 0
minAngle = 0
maxAngle = 180


In [16]:
# List available COM ports
ports = serial.tools.list_ports.comports()
portsList = []


In [17]:
for port_name in ports:
    portsList.append(str(port_name))
    print(str(port_name))


COM5 - Silicon Labs CP210x USB to UART Bridge (COM5)


In [19]:
# Select the COM port number
com = input("Select the COM port number: ")

# Verify and set the selected COM port
for i in range(len(portsList)):
    if portsList[i].startswith("COM" + str(com)):
        use = "COM" + str(com)
        print("Using port:", use)


Select the COM port number:  5


Using port: COM5


In [20]:
#Initialize some object
detector = htm.handDetector(detectionCon=0.7)

In [21]:
# Initialize camera feed
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# Set up serial communication with the ESP32
serialInst = serial.Serial()
serialInst.baudrate = 115200
serialInst.port = use
serialInst.open()

while True:
    success, img = cap.read()  # Capture the video frame
    img = detector.findHands(img)  # Detect hands in the frame

    # Find landmark positions of the hand
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        # Thumb tip
        x1, y1 = lmList[4][1], lmList[4][2]
        # Index finger tip
        x2, y2 = lmList[8][1], lmList[8][2]

        # Calculate the center between the thumb and index finger tips
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # Visualize the thumb and index tip along with the line between them
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        # Compute the length between thumb and index tips
        length = math.hypot(x2 - x1, y2 - y1)

        # Map the length to an angle between 0 and 180 degrees
        angle = np.interp(length, [6, 220], [minAngle, maxAngle])
        angle_bar = np.interp(length, [6, 220], [370, 10])

        # Debugging print statement (optional)
        print(f"Length: {length}, Angle: {int(angle)}")

        # Send the angle to ESP32 via serial
        serialInst.write(f"{int(angle)}\n".encode('utf-8'))

    # Display the angle bar
    cv2.rectangle(img, (50, 10), (85, 380), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(angle_bar)), (85, 380), (0, 255, 0), cv2.FILLED)

    # FPS calculation
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    # Display FPS on the frame
    cv2.putText(img, f'FPS: {int(fps)}', (500, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    
    # Show the video frame
    cv2.imshow("frame", img)

    # Exit the loop on ESC key press
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Escape key
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()
serialInst.close()

Length: 14.7648230602334, Angle: 7
Length: 12.08304597359457, Angle: 5
Length: 9.848857801796104, Angle: 3
Length: 8.06225774829855, Angle: 1
Length: 4.123105625617661, Angle: 0
Length: 9.055385138137417, Angle: 2
Length: 8.06225774829855, Angle: 1
Length: 10.0, Angle: 3
Length: 12.0, Angle: 5
Length: 9.055385138137417, Angle: 2
Length: 10.19803902718557, Angle: 3
Length: 11.180339887498947, Angle: 4
Length: 13.152946437965904, Angle: 6
Length: 11.180339887498947, Angle: 4
Length: 10.44030650891055, Angle: 3
Length: 12.36931687685298, Angle: 5
Length: 15.524174696260024, Angle: 8
Length: 16.278820596099706, Angle: 8
Length: 13.341664064126334, Angle: 6
Length: 18.439088914585778, Angle: 10
Length: 18.439088914585778, Angle: 10
Length: 32.0624390837628, Angle: 21
Length: 37.0540146272978, Angle: 26
Length: 38.2099463490856, Angle: 27
Length: 37.21558813185679, Angle: 26
Length: 40.44749683231337, Angle: 28
Length: 41.30375285612676, Angle: 29
Length: 47.67598976424087, Angle: 35
Length: